In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# Set up paths
BASE_PATH = '/workspace/COMP-3608---PROJECT'
DATA_PATH = os.path.join(BASE_PATH, 'data')
MODELS_PATH = os.path.join(BASE_PATH, 'models')
RESULTS_PATH = os.path.join(BASE_PATH, 'results')
FEATURE_ANALYSIS_PATH = os.path.join(RESULTS_PATH, 'feature_analysis')

In [3]:
# Create feature analysis directory if it doesn't exist
if not os.path.exists(FEATURE_ANALYSIS_PATH):
    os.makedirs(FEATURE_ANALYSIS_PATH)


In [4]:
# Load datasets
print("Loading datasets...")
diabetes_df = pd.read_csv(os.path.join(DATA_PATH, 'feature_engineering/diabetes_feature_engineering'))
stroke_df = pd.read_csv(os.path.join(DATA_PATH, 'feature_engineering/Stroke_feature_engineering'))
heart_df = pd.read_csv(os.path.join(DATA_PATH, 'feature_engineering/heart_feature_engineering'))


Loading datasets...


In [5]:
print(f"Diabetes dataset shape: {diabetes_df.shape}")
print(f"Stroke dataset shape: {stroke_df.shape}")
print(f"Heart Disease dataset shape: {heart_df.shape}")

Diabetes dataset shape: (100000, 12)
Stroke dataset shape: (5110, 20)
Heart Disease dataset shape: (920, 16)
